In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPool2D
from tensorflow.keras.models import Sequential

2021-09-25 03:34:33.926659: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
dataset_1 = pd.read_csv('creditcard.csv')

In [3]:
dataset_1.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# Data preprocessing
dataset_1.shape

(284807, 31)

In [5]:
dataset_1.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [6]:
dataset_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [7]:
dataset_1['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [8]:
# Balancing dataset
fraud=dataset_1[dataset_1['Class']==1]
non_fraud=dataset_1[dataset_1['Class']==0]

In [9]:
fraud.shape,non_fraud.shape

((492, 31), (284315, 31))

In [10]:
#random selection of samples
non_fraud_t = non_fraud.sample(n=492)

In [11]:
non_fraud_t.shape

(492, 31)

In [12]:
# merging dataset
dataset = fraud.append(non_fraud_t,ignore_index=True)

In [14]:
dataset['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [15]:
# matrix of features
x= dataset.drop(labels=['Class'],axis=1)

In [16]:
# dependent variable
y = dataset['Class']

In [17]:
x.shape,y.shape

((984, 30), (984,))

In [18]:
# Splitting dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [19]:
x_train.shape,x_test.shape

((787, 30), (197, 30))

In [21]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [22]:
x_train

array([[ 0.95535352,  0.03975587, -0.11145176, ..., -0.32484331,
         0.05170643, -0.19676419],
       [-0.81385863,  0.50715075, -0.11667159, ...,  0.4730311 ,
         0.60503684, -0.4590976 ],
       [-0.8501072 ,  0.19044323,  0.08291553, ...,  0.31074146,
         0.45159893, -0.45796052],
       ...,
       [ 0.89029408,  0.43270617, -0.32835062, ..., -0.01461624,
         0.12937687, -0.21746849],
       [ 1.45273947,  0.4449359 , -0.29676937, ...,  0.1443552 ,
         0.12242578, -0.44568955],
       [-1.56853994,  0.68679854, -0.65837348, ..., -0.14961862,
        -0.0887514 , -0.38713003]])

In [23]:
# because only 1 dim output
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [24]:
x_train=x_train.reshape(787,30,1)
x_test=x_test.reshape(197,30,1)

In [26]:
# Buiding Model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(filters=32,kernel_size=2,padding='same',activation='relu',input_shape=(30,1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.2))

model.add(tf.keras.layers.Conv1D(filters=64,kernel_size=2,padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 30, 32)            96        
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 32)            128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 15, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 15, 64)            4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 64)            256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 7, 64)            

In [28]:
opt=tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
history=model.fit(x_train,y_train,epochs=25,validation_data=(x_test,y_test))

2021-09-25 03:55:05.614839: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-25 03:55:05.696253: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299965000 Hz


Epoch 1/25
25/25 [==============================] - 3s 24ms/step - loss: 0.8613 - accuracy: 0.5594 - val_loss: 0.6436 - val_accuracy: 0.7665
Epoch 2/25
25/25 [==============================] - 0s 7ms/step - loss: 0.4735 - accuracy: 0.7598 - val_loss: 0.6127 - val_accuracy: 0.7970
Epoch 3/25
25/25 [==============================] - 0s 7ms/step - loss: 0.3823 - accuracy: 0.8582 - val_loss: 0.5875 - val_accuracy: 0.8376
Epoch 4/25
25/25 [==============================] - 0s 7ms/step - loss: 0.2840 - accuracy: 0.8983 - val_loss: 0.5540 - val_accuracy: 0.8528
Epoch 5/25
25/25 [==============================] - 0s 7ms/step - loss: 0.2796 - accuracy: 0.9015 - val_loss: 0.5214 - val_accuracy: 0.8731
Epoch 6/25
25/25 [==============================] - 0s 8ms/step - loss: 0.2363 - accuracy: 0.9298 - val_loss: 0.4813 - val_accuracy: 0.8883
Epoch 7/25
25/25 [==============================] - 0s 9ms/step - loss: 0.2473 - accuracy: 0.9093 - val_loss: 0.4394 - val_accuracy: 0.8934
Epoch 8/25
25/25 [=

In [33]:
# model prediction
y_pred = model.predict_classes(x_test)

In [36]:
# confusion metrics
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[102   4]
 [  7  84]]


In [39]:
print(accuracy_score(y_test,y_pred)*100,"%")

94.41624365482234 %
